In [18]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import importlib
import sys

# import validation module
sys.path.append('../')
validation = importlib.import_module('validation')

# import cleanair module
sys.path.append('../../containers')
cleanair = importlib.import_module('cleanair')
from cleanair.databases import DBReader
from cleanair.databases.tables import LAQNSite

In [19]:
name = 'basic'
experiment_data = '../experiment_data/'
exp = validation.util.load_experiment_from_directory(
    name,
    directory=experiment_data,
    secretfile='../../terraform/.secrets/db_secrets.json'
)
exp.update_model_data_list(update_train=False, update_test=True)


../experiment_data/basic/results/svgp_param0_data0/test_pred.pickle

predict df shape: (3888, 0)
NO2 mean shape: (3888,)
NO2 var shape: (3888,)

../experiment_data/basic/results/svgp_param1_data0/test_pred.pickle

predict df shape: (3888, 0)
NO2 mean shape: (3888,)
NO2 var shape: (3888,)

../experiment_data/basic/results/svgp_param2_data0/test_pred.pickle

predict df shape: (3888, 0)
NO2 mean shape: (3888,)
NO2 var shape: (3888,)

../experiment_data/basic/results/svgp_param3_data0/test_pred.pickle

predict df shape: (3888, 0)
NO2 mean shape: (3888,)
NO2 var shape: (3888,)


In [20]:
for model_data in exp.model_data_list:
    scores = validation.metrics.measure_scores_by_hour(
        model_data.normalised_pred_data_df, validation.metrics.get_metric_methods()
    )

    fig = px.line(scores, x=scores.index, y=scores.r2)
    fig.show()


In [21]:
fig = go.Figure()
i = 0
for model_data in exp.model_data_list:
    scores = validation.metrics.measure_scores_by_hour(
        model_data.normalised_pred_data_df, validation.metrics.get_metric_methods()
    )

    fig.add_trace(go.Scatter(x=scores.index, y=scores.mae, name='experiment {i}'.format(i=i)))
    if i > 0:
        break
    i += 1

fig.show()

In [22]:
def get_LAQN_sensor_info(secret_fp):

    db_reader = DBReader(secretfile=secret_fp)

    with db_reader.dbcnxn.open_session() as session:
        
        LAQN_table = session.query(LAQNSite)

        return pd.read_sql(LAQN_table.statement, LAQN_table.session.bind)

In [23]:
sensor_df = get_LAQN_sensor_info('../../terraform/.secrets/db_secrets.json')

In [54]:
model_data = exp.model_data_list[0]
pred_df = model_data.normalised_pred_data_df.copy()
print(pred_df.columns)
trace = dict(
    type='scattergeo',
    lon = pred_df['lon'],
    lat = pred_df['lat'],
    mode='markers'
)

lyt = dict(geo = dict(
    resolution=50,
    lonaxis=dict(showgrid=True),
#     showland=True, showocean=True,
#     showcountries=True,
    showrivers=True,
    projection = dict(scale=1)
))

fig = go.Figure(data = [trace], layout = lyt)
# fig = go.Figure(data=go.Scattergeo(
#         lon = pred_df['lon'],
#         lat = pred_df['lat']
# ))
# fig.update_layout(geo_scope='europe')
# print(fig)
# print(fig['layout'])
# fig['layout']['yaxis'].update(range=[50,51], autorange=False)
# fig.layout.geo.fitbounds = True
fig.update_geos(
    dict(
    lataxis_range=[51.25, 51.75], lonaxis_range=[-0.5, 0.2]
    ))
fig.show()

Index(['point_id', 'measurement_start_utc', 'source', 'location', 'in_london',
       'lon', 'lat', 'date_opened', 'date_closed', 'value_1000_flat',
       'value_500_flat', 'value_200_flat', 'value_100_flat', 'value_10_flat',
       'epoch', 'NO2', 'epoch_norm', 'lat_norm', 'lon_norm',
       'value_1000_flat_norm', 'NO2_mean', 'NO2_var'],
      dtype='object')


In [ ]:
"""

fig.update_geos(
    center=dict(lon=-30, lat=-30),
    projection_rotation=dict(lon=30, lat=30, roll=30),
    lataxis_range=[-50,20], lonaxis_range=[0, 200]
)
"""